In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
import time

In [ ]:
headers = {'Authorization': 'Token token=""'}
indicator_id = '460'
start_date = '2017-01-01T00:00:00Z'
end_date = '2018-12-31T24:00:00Z'
#0a3ec68a89579911bd7610e20df0f79c83f79b10275b19025aaf414136c91f9a

In [ ]:
def get(indicator: str, start_date: str, end_date: str):

    root = f'https://api.esios.ree.es/indicators/{indicator}?&start_date={start_date}&end_date={end_date}/'

    data = requests.get(root,
                        headers=headers
                       )
    print(data.status_code)
    json = data.json()
    
    return json

In [ ]:
def df(json):
    
    table = pd.DataFrame()
    
    print("Converting JSON to DataFrame...")
    for val in tqdm(json['indicator']['values']):
        single_df = pd.DataFrame({'datetime': val['datetime_utc'], 'value': val['value']}, index=[0])
        table = table.append(single_df, ignore_index=True)

    table.insert(0, 'name', json['indicator']['name'])
    table.insert(1, 'id', json['indicator']['id'])
    table.insert(4, 'update_timestamp', json['indicator']['values_updated_at'])
    
    table.rename(columns={'value': 'demand_forecast'}, inplace=True)
    print("Conversion completed")
    
    return table

In [ ]:
prueba = get(indicator_id, '2017-01-01T00:00:00Z', '2018-12-31T24:00:00Z')

In [ ]:
df = df(prueba)

In [ ]:
df

In [ ]:
df['datetime'].min()

In [12]:
speed_df = pd.read_csv("../DataOps_test/dataset_speed_optimization.csv")

In [13]:
speed_df

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900
...,...,...,...,...,...,...,...,...,...,...,...
49995,417755,Carlton Arms Hotel,160 East 25th Street,New York,NY,10010,40.74027,-73.98262,2.0,121.4900,120.4900
49996,222985,"The Solita Soho Hotel, an Ascend Hotel Collect...",159 Grand St,New York,NY,10013,40.71988,-73.99888,3.0,379.0300,149.0100
49997,211803,"Mandarin Oriental, New York",80 Columbus Circle at 60th Street,New York,NY,10023,40.76919,-73.98283,5.0,3993.4500,694.7334
49998,326792,Upper West Side Brownstone,110 West 81st Street,New York,NY,10024,40.78354,-73.97515,3.0,291.0000,290.0000


In [15]:
# Function t o a p p l y t o a l l rows i n t h e DataFrame

def function_to_apply(lat, lon):
    a = np.sin(lat/2)**2 + np.cos(lon) * np.sin(lon/2)**2
    
    return a

In [39]:
df = speed_df.copy()

start = time.time()

list_results = []
for i in range(0, len(df)):
    r = function_to_apply(df.iloc[i]['latitude'], df.iloc[i]['longitude'])
    list_results.append(r)
    
df['distance'] = list_results

elapsed_time_fl = (time.time() - start)
print(elapsed_time_fl)

11.258590936660767


In [35]:
df

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate,distance
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216,0.358182
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000,0.359048
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600,0.331206
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597,0.354301
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900,0.358671
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,417755,Carlton Arms Hotel,160 East 25th Street,New York,NY,10010,40.74027,-73.98262,2.0,121.4900,120.4900,1.062820
49996,222985,"The Solita Soho Hotel, an Ascend Hotel Collect...",159 Grand St,New York,NY,10013,40.71988,-73.99888,3.0,379.0300,149.0100,1.067107
49997,211803,"Mandarin Oriental, New York",80 Columbus Circle at 60th Street,New York,NY,10023,40.76919,-73.98283,5.0,3993.4500,694.7334,1.064133
49998,326792,Upper West Side Brownstone,110 West 81st Street,New York,NY,10024,40.78354,-73.97515,3.0,291.0000,290.0000,1.061944


In [38]:
df = speed_df.copy()

start = time.time()

df['distance'] = np.sin(df['latitude']/2)**2 + np.cos(df['longitude']) * np.sin(df['longitude']/2)**2
elapsed_time_fl = (time.time() - start)

elapsed_time_fl

0.004088401794433594

In [19]:
for i in tqdm(range(0, len(df))):
    

42.68971

In [27]:
df

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate,distance
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216,0.358182
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000,0.359048
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600,0.331206
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597,0.354301
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900,0.358671
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,417755,Carlton Arms Hotel,160 East 25th Street,New York,NY,10010,40.74027,-73.98262,2.0,121.4900,120.4900,1.062820
49996,222985,"The Solita Soho Hotel, an Ascend Hotel Collect...",159 Grand St,New York,NY,10013,40.71988,-73.99888,3.0,379.0300,149.0100,1.067107
49997,211803,"Mandarin Oriental, New York",80 Columbus Circle at 60th Street,New York,NY,10023,40.76919,-73.98283,5.0,3993.4500,694.7334,1.064133
49998,326792,Upper West Side Brownstone,110 West 81st Street,New York,NY,10024,40.78354,-73.97515,3.0,291.0000,290.0000,1.061944
